# Експорт YOLOv8n у ONNX (FP16)

У цьому ноутбуці ми:
- Встановимо потрібні бібліотеки (`ultralytics`, `onnx`, `onnxruntime`).
- Завантажимо ваги `yolov8n.pt` (завантажаться автоматично, якщо їх немає локально).
- Експортуємо модель у формат ONNX з напівточністю (FP16) і динамічними розмірами.
- Перевіримо експортований ONNX (структура та пробний запуск).

Примітки:
- FP16 ONNX краще виконувати на GPU/CPU з підтримкою FP16; на CPU можливі автоматичні перетворення типів.
- Шлях збереження можна змінити в змінній `save_dir`. За замовчуванням використаємо `trtexec_workspace/exports`.

In [1]:
# Завантажуємо модель YOLOv8n
from ultralytics import YOLO


model = YOLO('yolov8n.pt')

print("=== Початок експорту в TensorRT ===")

# Прямий експорт в TensorRT формат (.engine)
model.export(
    format='engine',      # Формат виходу - TensorRT
    device=0,            # Використовувати GPU 0
    half=True,          # FP16 (зменшена точність)
    workspace=4,         # 4GB пам'яті GPU
    verbose=True         # Детальний вивід
)

print("✅ Експорт завершено!")        # Використовуємо CPU


=== Початок експорту в TensorRT ===
Ultralytics 8.3.209  Python-3.10.18 torch-2.8.0+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


# Спростити модель

In [ ]:
import onnx
import onnxsim

model = onnx.load('model.onnx')
model_simp, check = onnxsim.simplify(model)
onnx.save(model_simp, 'model_simplified.onnx')
print('✅ Модель успішно спрощена')
print(f'Input: {model_simp.graph.input[0].name} - {[d.dim_value for d in model_simp.graph.input[0].type.tensor_type.shape.dim]}')
print(f'Output: {model_simp.graph.output[0].name} - {[d.dim_value for d in model_simp.graph.output[0].type.tensor_type.shape.dim]}')

In [ ]:
# Перевірка експортованого ONNX (структура та типи)

import onnx

import numpy as np

import onnxruntime as ort

from pathlib import Path



# Знайдемо останній експортований файл (Ultralytics повертає шлях або None)

search_dir = Path().resolve()

candidates = sorted(search_dir.rglob("*.onnx"))

assert candidates, "Не знайдено .onnx у цій директорії. Запустіть попередню клітинку."

onnx_file = str(candidates[-1])

print("ONNX файл:", onnx_file)



# 1) Перевірка валідності графа

model = onnx.load(onnx_file)

onnx.checker.check_model(model)

print("✅ ONNX пройшов перевірку onnx.checker")



# Порахувати кількість 16-бітних тензорів серед ініціалізаторів

fp16_inits = sum(1 for t in model.graph.initializer if t.data_type == onnx.TensorProto.FLOAT16)

fp32_inits = sum(1 for t in model.graph.initializer if t.data_type == onnx.TensorProto.FLOAT)

print(f"Ініціалізатори FP16: {fp16_inits}, FP32: {fp32_inits}")



# 2) Пробний інференс через ONNX Runtime (CPU)

sess = ort.InferenceSession(onnx_file, providers=["CPUExecutionProvider"])  # за потреби можна додати CUDA

input_name = sess.get_inputs()[0].name

print("Вхід:", sess.get_inputs()[0])

print("Виходи:", sess.get_outputs())



# Згенерувати випадковий вхід 1x3x640x640 у діапазоні [0,1]

dummy = np.random.rand(1, 3, 640, 640).astype(np.float32)  # runtime сам виконає приведення, якщо модель FP16

out = sess.run(None, {input_name: dummy})

print("Інференс успішний. К-сть виходів:", len(out))